In [4]:
!pip3 install bert-score rouge-score nltk


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached absl_py-2.2.2-py3-none-any.whl (135 kB)
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24951 sha256=a511ebc4f5bc8533dfc48e2810c407c618615e4ae62663c0d3a11d3800d24dcc
  Stored in directory: /home/rahul/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [11]:
import pandas as pd
from bert_score import score as bert_score
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
import nltk
# Load final outputs
df = pd.read_csv('./final_responses.csv')
references = df['counterspeech'].astype(str).tolist()
targets = df['generated_counterspeech'].astype(str).tolist()
nltk.download('wordnet')
# ---------- BERTScore ----------
P, R, F1 = bert_score(targets, references, lang='en', rescale_with_baseline=True)
bert_precision = P.mean().item()
bert_recall = R.mean().item()
bert_f1 = F1.mean().item()

# ---------- ROUGE ----------
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
r1s, r2s, rLs = [], [], []
for ref, hyp in zip(references, targets):
    scores = rouge.score(ref, hyp)
    r1s.append(scores['rouge1'].fmeasure)
    r2s.append(scores['rouge2'].fmeasure)
    rLs.append(scores['rougeL'].fmeasure)
rouge_1 = sum(r1s) / len(r1s)
rouge_2 = sum(r2s) / len(r2s)
rouge_L = sum(rLs) / len(rLs)

# ---------- METEOR ----------
# Tokenize references and hypotheses, then compute METEOR scores
meteors = [meteor_score([word_tokenize(ref)], word_tokenize(hyp)) 
           for ref, hyp in zip(references, targets)]

# Compute average METEOR score
meteor_avg = sum(meteors) / len(meteors)

# ---------- Print in Table Format ----------
print("\n📊 Final Evaluation Metrics:\n")
print(f"ROUGE-1 & {rouge_1:.4f} \\\\")
print(f"ROUGE-2 & {rouge_2:.4f} \\\\")
print(f"ROUGE-L & {rouge_L:.4f} \\\\")
print(f"METEOR & {meteor_avg:.4f} \\\\")
print(f"BERTScore (Precision) & {bert_precision:.4f} \\\\")
print(f"BERTScore (Recall) & {bert_recall:.4f} \\\\")
print(f"BERTScore (F1) & {bert_f1:.4f} \\\\")


[nltk_data] Downloading package wordnet to /home/rahul/nltk_data...
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



📊 Final Evaluation Metrics:

ROUGE-1 & 0.1764 \\
ROUGE-2 & 0.0297 \\
ROUGE-L & 0.1321 \\
METEOR & 0.1155 \\
BERTScore (Precision) & 0.2395 \\
BERTScore (Recall) & 0.1209 \\
BERTScore (F1) & 0.1799 \\


In [7]:
responses_csv = pd.read_csv('./final_responses.csv')
references = responses_csv['counterspeech']
targets = responses_csv['generated_counterspeech']
for ref,target in zip(references,targets):
    print(f'ref:{ref}\ntarget:{target}')

ref:The characteristics you are referring to are a strength, not weakness. spreading this kind of gender bias is exactly why the patriarchy needs to stop.
target:Women are not childlike and many of them work hard to take care of their families.
ref:Nobody apart from people who spread misinformation like this seem to have a problem with this.
target:Women are not childlike and many of them work hard to take care of their families.
ref:It is a question of survival and happiness to accept our differences and similarities and to work together.
target:Women are not childlike and many of them work hard to take care of their families.
ref:This is incorrect. jews could never be able to affect such a drastic change among the white people, given the amount of corrpution that goes on in higher positions of power.
target:What evidence supports the claim that minorities are voting with the intent to strip wealth and power from white people, organized by jws?
ref:Where is your evidence?
target:What 

In [16]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import re

def evaluate_counterspeech_from_csv(csv_path, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    Evaluate category accuracy of counterspeech intents in a CSV using a judge language model.
    Scores each counterspeech for all intents (1-10), predicts the highest-scoring intent,
    and checks if it's among valid intents for the hate speech.

    Args:
        csv_path (str): Path to the CSV file containing hate speech and counterspeech.
        device (str): Device to run the judge model on ('cuda' or 'cpu').

    Returns:
        float: Category accuracy of predicted intents vs. valid intents per hate speech.
    """
    # Define intent categories
    categories = {
        'Informative': 0,
        'Questioning': 1,
        'Denouncing': 2,
        'Positive': 3
    }

    # Load judge model and tokenizer
    judge_tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
    judge_model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small").to(device)
    judge_model.eval()
    print("Judge model ready!")

    # Load CSV
    df = pd.read_csv(csv_path)

    # Group by hatespeech to collect all valid intents
    intent_map = df.groupby('hatespeech')['csType'].apply(lambda x: set(str(i).capitalize() for i in x)).to_dict()

    total_samples = 0
    correct_predictions = 0

    # Progress bar for evaluation
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Evaluating Counterspeech"):
        hatespeech = str(row['hatespeech'])
        counterspeech = str(row['counterspeech'])
        true_intent = str(row['csType']).capitalize()  # Current row’s intent

        # Skip if intent is not in categories
        if true_intent not in categories:
            continue

        # Get valid intents for this hate speech
        valid_intents = intent_map.get(hatespeech, set())

        intent_scores = {}

        # Score counterspeech for each intent
        for intent in categories:
            # Construct prompt
            prompt = (
                f"Hate speech: {hatespeech}\n"
                f"Proposed counterspeech (intended as {intent}): {counterspeech}\n\n"
                f"On a scale of 1 to 10, how appropriate and effective is this counterspeech in addressing the hate speech as an {intent} response? "
                f"Respond with a single number between 1 and 10."
            )

            # Encode and generate response
            judge_input = judge_tokenizer.encode(prompt, return_tensors='pt').to(device)
            with torch.no_grad():
                output_ids = judge_model.generate(
                    judge_input,
                    max_new_tokens=10,  # Enough for a number
                    pad_token_id=judge_tokenizer.eos_token_id
                )
            score_text = judge_tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

            # Extract numerical score
            try:
                score = float(re.search(r'\d+(\.\d+)?', score_text).group(0))
                score = max(0, min(score, 10))  # Clamp between 0 and 10
            except (AttributeError, ValueError):
                score = 0  # Fallback if no valid number

            intent_scores[intent] = score

        # Predict the best intent
        best_intent = max(intent_scores, key=intent_scores.get)

        # Check if prediction is correct (in valid intents for this hate speech)
        if best_intent in valid_intents:
            correct_predictions += 1
        total_samples += 1

    # Compute final accuracy
    final_accuracy = correct_predictions / total_samples if total_samples else 0
    print(f"\nFinal Category Accuracy using Judge LM (highest-scoring intent in valid intents): {final_accuracy:.4f}")
    return final_accuracy

# Example usage
if __name__ == "__main__":
    csv_path = "./IntentConanv2/test.csv"  # Adjust to your CSV path
    accuracy = evaluate_counterspeech_from_csv(csv_path)

Judge model ready!


Evaluating Counterspeech: 100%|██████████| 2971/2971 [02:20<00:00, 21.08it/s]


Final Category Accuracy using Judge LM (highest-scoring intent in valid intents): 0.7540
